In [5]:
import numpy as np
import math 

from pydrake.common.containers import namedview
from pydrake.systems.framework import (BasicVector,
                                       LeafSystem,
                                       PublishEvent,
                                       AbstractValue,
                                       UnrestrictedUpdateEvent,
                                       WitnessFunctionDirection)

CompassGaitState = namedview('CompassGaitState', ['stance', 'swing', 'stancedot', 'swingdot'])

class CompassGait(LeafSystem):
    def __init__(self, slope, mass_hip):
        LeafSystem.__init__(self)
        
        self.DeclareContinuousState(BasicVector(np.zeros(4)), 2, 2, 0)
        self.DeclareDiscreteState(1)
        
        left_stance = True 
        self.DeclareAbstractState(AbstractValue.Make(left_stance))
        
        self.DeclareVectorOutputPort("minimal", BasicVector(4), self.MinimalStateOut)
        self.DeclareVectorOutputPort("floatingbase", BasicVector(14), self.FloatingBaseStateOut)
        
        self.gravity = 9.81
        self.slope = slope
        
        self.mass_leg = 5
        self.mass_hip = mass_hip
        self.length_leg = 1.0 
        self.center_of_mass_leg = 0.5
        self.distance = self.length_leg - self.center_of_mass_leg
        
        self.vsw_des = 0 
        self.u_count = 0
        self.add = False 
        
        self.collision_witness = self.MakeWitnessFunction(
            "foot_collision", WitnessFunctionDirection.kPositiveThenNonPositive, 
            self.foot_collision, UnrestrictedUpdateEvent(self.collision_dynamics))
        
    def get_toe_position(self, context):
        return context.get_discrete_state(0).GetAtIndex(0)
    
    def set_toe_position(self, state, value):
#         print ("set_toe_position")
        state.get_discrete_state().get_mutable_vector(0).get_mutable_value()[0]=value
        
    def left_leg_is_stance(self, context):
        return context.get_discrete_state(1).GetAtIndex(0)
    
    def set_left_leg_is_stance(self, state, value):
#         print ("set_left_leg_is_stance")
        state.get_discrete_state().get_mutable_vector(1).get_mutable_value()[0]=value
        
    def foot_collision(self, context):
#         print ("foot_collision")
        c = CompassGaitState(context.get_continuous_state_vector().CopyToVector())
        collision = 2*self.slope - c.stance - c.swing
        return max(collision, c.swing - c.stance)
    
    def add_torque(self, context):
        c = CompassGaitState(context.get_continuous_state_vector().CopyToVector())
        
        eps = 0.05
        slope = self.slope
        st_r = c.stance + (np.pi/2 - slope) 
        sw_r = c.swing + (np.pi/2 - slope)
        vst = c.stancedot
        vsw = c.swingdot 

        tperp = abs((np.pi/2 - st_r) / vst) 
#         print (tperp, sw_r + vsw * tperp - 0.5*self.gravity*math.sin(c.swing)*tperp**2)

        self.vsw_des = vsw 

        if sw_r + vsw * tperp - 0.5*self.gravity*math.sin(c.swing)*tperp**2 > np.pi/2 - eps:
#             print ('add torque')
            self.vsw_des = (np.pi/2 - eps - sw_r + 0.5*self.gravity*math.sin(c.swing)*tperp**2) / tperp
            self.add = True 
    
    def collision_dynamics(self, context, event, state):
#         print ("collision_dynamics")
#         print (type(context), type(state))
        self.new_stance = True 
        self.add_torque(context)
        
        c = CompassGaitState(context.get_continuous_state_vector().CopyToVector())
        next_c = CompassGaitState(context.get_mutable_continuous_state_vector().CopyToVector())
        
        m = self.mass_leg
        mh = self.mass_hip
        a = self.distance
        b = self.center_of_mass_leg
        l = self.length_leg
        cst = math.cos(c.stance)
        csw = math.cos(c.swing)
        hip_angle = c.swing - c.stance
        ch = math.cos(hip_angle)
        sh2 = math.sin(hip_angle / 2)
        sst = math.sin(c.stance)
        ssw = math.sin(c.swing)
      
        
        M = np.array([[2*m + mh, 0, (m * a + m * l + mh * l) * cst, -m * b * csw], 
                     [0, 2 * m + mh, -(m * a + m * l + mh * l) * sst, m * b * ssw], 
                     [(m * a + m * l + mh * l) * cst, -(m * a + m * l + mh * l) * sst, 
                      m * a * a + (m + mh) * l * l,    -m * l * b * ch], 
                     [-m * b * csw, m * b * ssw, -m * l * b * ch, m * b * b]])
        J = np.array([[1, 0, l * cst,  -l * csw], 
                      [0, 1, -l * sst, l * ssw]])
        
        V_pre = np.array([0, 0, c.stancedot, c.swingdot])
        M_inv = np.linalg.inv(M)
    
        V_post = V_pre - M_inv.dot(J.T).dot(np.linalg.inv(J.dot(M_inv).dot(J.T))).dot(J).dot(V_pre)
        
        next_c.stance = c.swing
        next_c.swing = c.stance
        next_c.stancedot = V_post[3]
        next_c.swingdot = V_post[2]
        
        self.set_toe_position(state, self.get_toe_position(context) - 2 * l * sh2)
#         self.set_left_leg_is_stance(state, abs(self.left_leg_is_stance(context)-1))
        
        state.get_mutable_continuous_state().get_mutable_vector().SetFromVector(next_c[:])
        
    def MinimalStateOut(self, context, output):
#         print ("MinimalStateOut")
        x = context.get_continuous_state_vector().CopyToVector()
        y = output.SetFromVector(x)
        
    def FloatingBaseStateOut(self, context, output):
#         print ("FloatingBaseStateOut")
        c = CompassGaitState(context.get_continuous_state_vector().CopyToVector())
        toe = self.get_toe_position(context)
        left_stance = True #self.left_leg_is_stance(context)
        
        m = self.mass_leg
        mh = self.mass_hip
        a = self.distance
        b = self.center_of_mass_leg
        l = self.length_leg
        cs = math.cos(self.slope)
        ss = math.sin(self.slope)
        cst = math.cos(c.stance)
        csw = math.cos(c.swing)
        hip_angle = c.swing - c.stance
        ch = math.cos(hip_angle)
        sst = math.sin(c.stance)
        ssw = math.sin(c.swing)
        
        output.SetAtIndex(0, toe * cs + l * sst)
        output.SetAtIndex(1, 0)
        output.SetAtIndex(2, -toe * ss + l * cst)
        
        left = c.stance if left_stance else c.swing 
        right = c.swing if left_stance else c.stance
        
        output.SetAtIndex(3, 0)
        output.SetAtIndex(4, left)
        output.SetAtIndex(5, 0)
        
        output.SetAtIndex(6, right - left)
        
        output.SetAtIndex(7, c.stancedot * l * cst)
        output.SetAtIndex(8, 0)
        output.SetAtIndex(9, -c.stancedot * l * sst)
        
        leftdot = c.stancedot if left_stance else c.swingdot
        rightdot = c.swingdot if left_stance else c.stancedot 

        output.SetAtIndex(10, 0)
        output.SetAtIndex(11, leftdot)
        output.SetAtIndex(12, 0)
        
        output.SetAtIndex(13, rightdot - leftdot)
        
    def DynamicsBiasTerm(self, context):
        c = CompassGaitState(context.get_continuous_state_vector().CopyToVector())
        
        m = self.mass_leg
        mh = self.mass_hip
        a = self.distance
        b = self.center_of_mass_leg
        l = self.length_leg
        g = self.gravity
        s_diff =  math.sin(c.stance - c.swing)
        sst = math.sin(c.stance)
        ssw = math.sin(c.swing)
        vst = c.stancedot
        vsw = c.swingdot
        
        bias = np.array([-m * l * b * vsw * vsw * s_diff - (mh * l + m * (a + l)) * g * sst, 
                         m * l * b * vst * vst * s_diff + m * b * g * ssw])
        
        return bias 
    
    def ControlMatrix(self, context):
        u = 0
        B = np.array([0, -1])
        
        k = self.DoCalcKineticEnergy(context)
        c = CompassGaitState(context.get_continuous_state_vector().CopyToVector())
#         print (k) 
        
        if k < 20 and c.stancedot > 0: 
            u = -1.0
        if k < 20 and c.stancedot < 0: 
            u = 1.0 
        
#         c = CompassGaitState(context.get_continuous_state_vector().CopyToVector())
#         vsw = c.swingdot 
        
#         if self.add: 
#             u = self.vsw_des - vsw
#             self.u_count += 1 
#         if self.u_count >= 50: 
#             self.add = False 
#             self.u_count = 0 
            
#         print (u) 
        
        return B*u 
        
    def MassMatrix(self, context):
        c = CompassGaitState(context.get_continuous_state_vector().CopyToVector())
        
        m = self.mass_leg
        mh = self.mass_hip
        a = self.distance
        b = self.center_of_mass_leg
        l = self.length_leg
        c_diff = math.cos(c.swing - c.stance)
        
        M = np.array([[mh * l * l + m * (l * l + a * a), -m * l * b * c_diff], 
                      [-m * l * b * c_diff, m * b * b]])
        
        return M 
        
    def DoGetWitnessFunctions(self, context):
#         print ("DoGetWitnessFunctions")
        return [self.collision_witness]

    def DoCalcTimeDerivatives(self, context, derivatives):
#         print ("DoCalcTimeDerivatives")
        c = CompassGaitState(context.get_continuous_state_vector().CopyToVector())
        cdot = CompassGaitState(np.zeros(4))
        cdot[0:2] = c[2:4]
        
        M = self.MassMatrix(context)
        bias = self.DynamicsBiasTerm(context)
        B = self.ControlMatrix(context)
        
        a = -np.linalg.inv(M).dot(bias - B) 

        cdot.stancedot = a[0]
        cdot.swingdot = a[1]

        derivatives.get_mutable_vector().SetFromVector(cdot[:])
        
    def DoCalcKineticEnergy(self, context): 
        c = CompassGaitState(context.get_continuous_state_vector().CopyToVector())
        
        m = self.mass_leg
        mh = self.mass_hip
        a = self.distance
        b = self.center_of_mass_leg
        l = self.length_leg
        vst = c.stancedot
        vsw = c.swingdot 
        
        return 0.5 * (mh * l * l + m * a * a) * vst * vst + 0.5 * m * (l * l * vst * vst + b * b * vsw * vsw) - \
            m * l * b * vst * vsw * math.cos(c.swing - c.stance)
        
    def DoCalcPotentialEnergy(self, context): 
        c = CompassGaitState(context.get_continuous_state_vector().CopyToVector())
        
        m = self.mass_leg
        mh = self.mass_hip
        a = self.distance
        b = self.center_of_mass_leg
        l = self.length_leg
        g = self.gravity 
        
        y_toe = -self.get_toe_position(context) * math.sin(self.slope)
        y_hip = y_toe + l * math.cos(c.stance)
        
        return m * g * (y_toe + a * math.cos(c.stance)) + mh * g * y_hip + m * g * (y_hip - b * math.cos(c.swing))

In [ ]:
import argparse
import math
import numpy as np
import matplotlib.pyplot as plt

from pydrake.all import (Box,
                         DiagramBuilder,
                         FindResourceOrThrow,
                         FloatingBaseType,
                         Isometry3,
                         RigidBodyTree,
                         RigidBodyPlant,
                         SignalLogger,
                         Simulator,
                         VisualElement)
# from pydrake.examples.compass_gait import (CompassGait, CompassGaitParams)
from underactuated import (PlanarRigidBodyVisualizer)
from IPython.display import HTML

tree = RigidBodyTree("CompassGait.urdf", FloatingBaseType.kRollPitchYaw)

# slope = 0.0525 
mass_hip = 10
slope = 0.01
# mass_hip = 10

R = np.identity(3)
R[0, 0] = math.cos(slope)
R[0, 2] = math.sin(slope)
R[2, 0] = -math.sin(slope)
R[2, 2] = math.cos(slope)
X = Isometry3(rotation=R, translation=[0, 0, -5.])
color = np.array([0.9297, 0.7930, 0.6758, 1])
tree.world().AddVisualElement(VisualElement(Box([100., 1., 10.]), X, color))
tree.compile()

builder = DiagramBuilder()

compass_gait = builder.AddSystem(CompassGait(slope, mass_hip))

logger = builder.AddSystem(SignalLogger(14))
logger.DeclarePeriodicPublish(0.033333, 0.0)
builder.Connect(compass_gait.get_output_port(1), logger.get_input_port(0))

vis = PlanarRigidBodyVisualizer(tree, xlim=[-1., 5.], ylim=[-1., 2.], figsize_multiplier=2)

diagram = builder.Build()
simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)

context = simulator.get_mutable_context()
diagram.Publish(context)  # draw once to get the window open
context.SetAccuracy(1e-4)
context.SetContinuousState([0., 0., 0.4, -2.])
# context.SetContinuousState([0., 0., 0.2, -1.])

simulator.StepTo(5.0)

ani = vis.animate(logger, 60, repeat=True)
plt.close(vis.fig)
HTML(ani.to_html5_video())